In [62]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
sns.set_style("whitegrid")
import fitdecode
import csv
import os


# Data Cleaning

In [45]:
#Set the file path for th fit files
folder_path = "C:/Users/tedjo/Downloads/Beyond Words/Football Garmin"

#load all the files to a dataframe
import os
import pandas as pd

dfs = []
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        df = pd.read_csv(os.path.join(folder_path, filename),low_memory= False)
        dfs.append(df)

df = pd.concat(dfs, ignore_index=True)      

In [46]:
# Filter the datafram to include the Data and record from the Type and Message columns
filtered_df = df[(df['Type'] == 'Data') & (df['Message'] == 'record')].copy()
filtered_df

,Type,Local Number,Message,Field 1,Value 1,Units 1,Field 2,Value 2,Units 2,Field 3,...,cycle_length16,enhanced_speed,cadence,heart_rate,temperature,fractional_cadence,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1
30,Data,12.0,record,timestamp,1071265463,s,distance,0.0,m,enhanced_altitude,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34,Data,12.0,record,timestamp,1071265465,s,distance,0.0,m,enhanced_altitude,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,Data,12.0,record,timestamp,1071265467,s,distance,0.0,m,enhanced_altitude,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38,Data,12.0,record,timestamp,1071265468,s,distance,0.0,m,enhanced_altitude,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40,Data,12.0,record,timestamp,1071265469,s,distance,0.0,m,enhanced_altitude,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225993,Data,12.0,record,timestamp,1060380329.0,s,position_lat,614543187.0,semicircles,position_long,...,NaN,NaN,NaN,NaN,NaN,NaN,202812.0,179631.0,156450.0,133269.0
225994,Data,0.0,record,timestamp,1060380333.0,s,position_lat,614542425.0,semicircles,position_long,...,NaN,NaN,NaN,NaN,NaN,NaN,202813.0,179632.0,156451.0,133270.0
225995,Data,0.0,record,timestamp,1060380338.0,s,position_lat,614541085.0,semicircles,position_long,...,NaN,NaN,NaN,NaN,NaN,NaN,202814.0,179633.0,156452.0,133271.0
225996,Data,0.0,record,timestamp,1060380339.0,s,position_lat,614540997.0,semicircles,position_long,...,NaN,NaN,NaN,NaN,NaN,NaN,202815.0,179634.0,156453.0,133272.0


In [47]:
#Export the filtered dataframe to a csv file
filtered_df.to_csv('C:/Users/tedjo/Downloads/Beyond Words/Football Garmin/filtered_df.csv')


In [48]:
import subprocess

#Define the command as as a string
command = 'python clean_garmin.py -i filtered_df.csv -o cleaned_df.csv'

process = subprocess.Popen(command, shell = True, stdout = subprocess.PIPE)
#Print the output in real-time 
for line in iter(process.stdout.readline, b''):
    print(line.decode('utf-8').strip())

#Wait for the process to finish
print('Command Excetued return code is:',process.returncode)


Reading Data...
Loaded in all 139,086 rows of unclean data...
Filtering Down to Data...
Pivoting Table...
Cleaning Timestamps...
Command Excetued return code is: None


In [49]:
#Import the cleaned csv file to a dataframe
clean_df = pd.read_csv('C:/Users/tedjo/Downloads/Beyond Words/Football Garmin/cleaned_df.csv')


In [50]:
#Drop the  columns we do no need for this assessment
clean_df = clean_df.drop(columns=['Unnamed: 0','cycle_length16','cadence','fractional_cadence'],axis=1)

#To find the position_lon and lat we will divide the position_lat and long by 11930465.4806
clean_df['position_lat'] = clean_df['position_lat']/11930465.4806
clean_df['position_long'] = clean_df['position_long']/11930465.4806
clean_df['distance'].isnull().sum()
clean_df.fillna(0, inplace=True)
clean_df.head()

,timestamp,distance,position_lat,enhanced_altitude,position_long,enhanced_speed,heart_rate,temperature
0,2023-07-31 21:03:25,0.00,51.510145,29.2,-0.399707,0.355,114.0,28.0
1,2023-07-31 21:03:27,0.00,0.000000,29.2,0.000000,0.000,115.0,28.0
2,2023-07-31 21:03:36,7.36,51.510195,28.6,-0.399767,0.868,117.0,28.0
3,2023-07-31 21:03:39,9.91,51.510216,28.6,-0.399780,1.037,114.0,28.0
4,2023-07-31 21:03:43,13.39,51.510247,28.6,-0.399773,0.804,113.0,28.0


In [51]:
clean_df['timestamp'] = pd.to_datetime(clean_df['timestamp'])
time_played = clean_df.groupby(clean_df['timestamp'].dt.day)['timestamp'].apply(lambda x: x.max() - x.min())

clean_df = pd.merge(clean_df, time_played, left_on=clean_df['timestamp'].dt.day, right_index=True, how='left')
clean_df


,timestamp_x,distance,position_lat,enhanced_altitude,position_long,enhanced_speed,heart_rate,temperature,timestamp_y
0,2023-07-31 21:03:25,0.00,51.510145,29.2,-0.399707,0.355,114.0,28.0,0 days 01:08:13
1,2023-07-31 21:03:27,0.00,0.000000,29.2,0.000000,0.000,115.0,28.0,0 days 01:08:13
2,2023-07-31 21:03:36,7.36,51.510195,28.6,-0.399767,0.868,117.0,28.0,0 days 01:08:13
3,2023-07-31 21:03:39,9.91,51.510216,28.6,-0.399780,1.037,114.0,28.0,0 days 01:08:13
4,2023-07-31 21:03:43,13.39,51.510247,28.6,-0.399773,0.804,113.0,28.0,0 days 01:08:13
...,...,...,...,...,...,...,...,...,...
21600,2023-12-11 23:11:38,5719.82,51.510204,-100.8,-0.400383,0.634,86.0,14.0,91 days 02:16:19
21601,2023-12-11 23:11:47,5727.32,51.510188,-100.8,-0.400278,0.774,82.0,14.0,91 days 02:16:19
21602,2023-12-11 23:11:50,5729.89,51.510182,-101.0,-0.400243,0.616,85.0,14.0,91 days 02:16:19
21603,2023-12-11 23:11:57,5734.91,51.510177,-101.0,-0.400173,0.644,82.0,14.0,91 days 02:16:19


In [52]:
clean_df.rename(columns={'timestamp_x':'timestamp','timestamp_y':'time_played'}, inplace=True)
clean_df.head()

,timestamp,distance,position_lat,enhanced_altitude,position_long,enhanced_speed,heart_rate,temperature,time_played
0,2023-07-31 21:03:25,0.00,51.510145,29.2,-0.399707,0.355,114.0,28.0,0 days 01:08:13
1,2023-07-31 21:03:27,0.00,0.000000,29.2,0.000000,0.000,115.0,28.0,0 days 01:08:13
2,2023-07-31 21:03:36,7.36,51.510195,28.6,-0.399767,0.868,117.0,28.0,0 days 01:08:13
3,2023-07-31 21:03:39,9.91,51.510216,28.6,-0.399780,1.037,114.0,28.0,0 days 01:08:13
4,2023-07-31 21:03:43,13.39,51.510247,28.6,-0.399773,0.804,113.0,28.0,0 days 01:08:13


In [53]:
clean_df['time_played'] = clean_df['time_played'].astype(str)
clean_df
clean_df['time_played'] = clean_df['time_played'].apply(lambda x: x.split()[-1])
clean_df


,timestamp,distance,position_lat,enhanced_altitude,position_long,enhanced_speed,heart_rate,temperature,time_played
0,2023-07-31 21:03:25,0.00,51.510145,29.2,-0.399707,0.355,114.0,28.0,01:08:13
1,2023-07-31 21:03:27,0.00,0.000000,29.2,0.000000,0.000,115.0,28.0,01:08:13
2,2023-07-31 21:03:36,7.36,51.510195,28.6,-0.399767,0.868,117.0,28.0,01:08:13
3,2023-07-31 21:03:39,9.91,51.510216,28.6,-0.399780,1.037,114.0,28.0,01:08:13
4,2023-07-31 21:03:43,13.39,51.510247,28.6,-0.399773,0.804,113.0,28.0,01:08:13
...,...,...,...,...,...,...,...,...,...
21600,2023-12-11 23:11:38,5719.82,51.510204,-100.8,-0.400383,0.634,86.0,14.0,02:16:19
21601,2023-12-11 23:11:47,5727.32,51.510188,-100.8,-0.400278,0.774,82.0,14.0,02:16:19
21602,2023-12-11 23:11:50,5729.89,51.510182,-101.0,-0.400243,0.616,85.0,14.0,02:16:19
21603,2023-12-11 23:11:57,5734.91,51.510177,-101.0,-0.400173,0.644,82.0,14.0,02:16:19


In [54]:
clean_df['time_played'] = pd.to_timedelta(clean_df['time_played']).astype(str).str.extract(r'(\d+:\d+:\d+)')
clean_df.head()


,timestamp,distance,position_lat,enhanced_altitude,position_long,enhanced_speed,heart_rate,temperature,time_played
0,2023-07-31 21:03:25,0.00,51.510145,29.2,-0.399707,0.355,114.0,28.0,01:08:13
1,2023-07-31 21:03:27,0.00,0.000000,29.2,0.000000,0.000,115.0,28.0,01:08:13
2,2023-07-31 21:03:36,7.36,51.510195,28.6,-0.399767,0.868,117.0,28.0,01:08:13
3,2023-07-31 21:03:39,9.91,51.510216,28.6,-0.399780,1.037,114.0,28.0,01:08:13
4,2023-07-31 21:03:43,13.39,51.510247,28.6,-0.399773,0.804,113.0,28.0,01:08:13


In [55]:
clean_df['time_played'] = pd.to_datetime(clean_df['time_played']).dt.time

C:\Users\tedjo\AppData\Local\Temp\ipykernel_1868\491739479.py:1: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



In [56]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21605 entries, 0 to 21604
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   timestamp          21605 non-null  datetime64[ns]
 1   distance           21605 non-null  float64       
 2   position_lat       21605 non-null  float64       
 3   enhanced_altitude  21605 non-null  float64       
 4   position_long      21605 non-null  float64       
 5   enhanced_speed     21605 non-null  float64       
 6   heart_rate         21605 non-null  float64       
 7   temperature        21605 non-null  float64       
 8   time_played        21605 non-null  object        
dtypes: datetime64[ns](1), float64(7), object(1)
memory usage: 1.5+ MB


In [57]:
#To find the speed, I divided the first value 0.355 by 1.3(the first value from the activity on my app to find 1 enhanced speed)
#Then I divided the enhanced speed by the value of 1
clean_df['Speed(km/h)'] = clean_df['enhanced_speed'] * (1.3/0.355)
clean_df['Speed(km/h)'] = clean_df['Speed(km/h)'].round(1)
#Move teh speed column to the seventh column and drop enhanced speed
clean_df = clean_df[['timestamp','position_lat','position_long','distance','Speed(km/h)','enhanced_altitude','heart_rate','time_played']]
clean_df.head()

,timestamp,position_lat,position_long,distance,Speed(km/h),enhanced_altitude,heart_rate,time_played
0,2023-07-31 21:03:25,51.510145,-0.399707,0.00,1.3,29.2,114.0,01:08:13
1,2023-07-31 21:03:27,0.000000,0.000000,0.00,0.0,29.2,115.0,01:08:13
2,2023-07-31 21:03:36,51.510195,-0.399767,7.36,3.2,28.6,117.0,01:08:13
3,2023-07-31 21:03:39,51.510216,-0.399780,9.91,3.8,28.6,114.0,01:08:13
4,2023-07-31 21:03:43,51.510247,-0.399773,13.39,2.9,28.6,113.0,01:08:13


In [58]:
clean_df['Date'] = clean_df['timestamp'].dt.date
clean_df['Time'] = clean_df['timestamp'].dt.time
clean_df.drop(columns=['timestamp'],axis=1,inplace=True)
first_column = clean_df.pop('Date')
second_column = clean_df.pop('Time')
clean_df.insert(0, 'Date', first_column)
clean_df.insert(1, 'Time', second_column)
clean_df.rename(columns= {'distance':'Cumulative_Distance(m)'},inplace=True)
clean_df.head()

C:\Users\tedjo\AppData\Local\Temp\ipykernel_1868\1907461684.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Date,Time,position_lat,position_long,Cumulative_Distance(m),Speed(km/h),enhanced_altitude,heart_rate,time_played
0,2023-07-31,21:03:25,51.510145,-0.399707,0.00,1.3,29.2,114.0,01:08:13
1,2023-07-31,21:03:27,0.000000,0.000000,0.00,0.0,29.2,115.0,01:08:13
2,2023-07-31,21:03:36,51.510195,-0.399767,7.36,3.2,28.6,117.0,01:08:13
3,2023-07-31,21:03:39,51.510216,-0.399780,9.91,3.8,28.6,114.0,01:08:13
4,2023-07-31,21:03:43,51.510247,-0.399773,13.39,2.9,28.6,113.0,01:08:13


# Data Visualisations

A heatmap of positional play

Directional movement on the field

Speed and heart rate



Heat map of position


In [65]:
fig = make_subplots(rows=2, cols=2, subplot_titles=("Distance Covered", "Heart Rate", "Speed", "Altitude"))

fig.add_trace(
    go.Scatter(x=clean_df['Date'], y=clean_df['Cumulative_Distance(m)'], name="Distance Covered"),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=clean_df['Date'], y=clean_df['heart_rate'], name="Heart Rate"),
    row=1, col=2
)

fig.add_trace(
    go.Scatter(x=clean_df['Date'], y=clean_df['Speed(km/h)'], name="Speed"),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=clean_df['Date'], y=clean_df['enhanced_altitude'], name="Altitude"),
    row=2, col=2
)

fig.update_layout(height=700, width=1000, title_text="Football Activity")

fig.show()

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Create the Dash application
app = dash.Dash(__name__)

# Define the layout of the application
app.layout = html.Div([
    html.H1("Filter Data by Date"),
    dcc.Dropdown(
        id='date-dropdown',
        options=[{'label': str(date), 'value': date} for date in clean_df['Date'].unique()],
        value=clean_df['Date'].min(),
    ),
    dcc.Dropdown(
        id='data-filter',
        options=[
            {'label': 'Speed', 'value': 'Speed(km/h)'},
            {'label': 'Altitude', 'value': 'enhanced_altitude'},
            {'label': 'Distance Covered', 'value': 'Cumulative_Distance(m)'},
            {'label': 'Heart Rate', 'value': 'heart_rate'}
        ],
        value=['Speed(km/h)', 'enhanced_altitude', 'Cumulative_Distance(m)', 'heart_rate'],
        multi=True
    ),
    html.Div(id='filtered-data')
])

# Define the callback function to update the filtered data
@app.callback(
    Output('filtered-data', 'children'),
    [Input('date-dropdown', 'value'),
     Input('data-filter', 'value')]
)
def update_filtered_data(selected_date, data_filter):
    filtered_df = clean_df[
        clean_df['Date'] == selected_date
    ]
    
    # Filter the selected columns
    filtered_df = filtered_df[data_filter]
    
    return html.Table([
        html.Thead(html.Tr([html.Th(col) for col in filtered_df.columns])),
        html.Tbody([
            html.Tr([
                html.Td(str(filtered_df.iloc[i][col])) for col in filtered_df.columns
            ]) for i in range(len(filtered_df))
        ])
    ])

# Run the Dash application
if __name__ == '__main__':
    app.run_server(debug=True)


In [79]:
clean_df.to_excel('garmin_data.xlsx')